# Constrained Bayesian optimisation

In [1]:
import torch
from nubo.acquisition import ExpectedImprovement, UpperConfidenceBound
from nubo.models import GaussianProcess, fit_gp
from nubo.optimisation import slsqp
from nubo.test_functions import Hartmann6D
from nubo.utils import gen_inputs
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood


# test function
func = Hartmann6D(minimise=False)
dims = func.dims
bounds = func.bounds

# training data
x_train = gen_inputs(num_points=dims*5,
                     num_dims=dims,
                     bounds=bounds,
                     seed=1)
y_train = func(x_train)

# Bayesian optimisation loop
iters = 40

for iter in range(iters):
    
    # specify Gaussian process
    likelihood = GaussianLikelihood()
    gp = GaussianProcess(x_train, y_train, likelihood=likelihood)
    mll = ExactMarginalLogLikelihood(likelihood=likelihood, model=gp)
    
    # fit Gaussian process
    fit_gp(x_train, y_train, gp=gp, likelihood=likelihood, mll=mll, lr=0.1, steps=200)

    # specify acquisition function
    # acq = ExpectedImprovement(gp=gp, y_best=torch.max(y_train))
    acq = UpperConfidenceBound(gp=gp, beta=1.96**2)

    # define constraints
    cons = ({'type': 'ineq', 'fun': lambda x: 0.5 - x[0] - x[1] },
            {'type': 'eq', 'fun': lambda x: 1.2442 - x[3] - x[4] - x[5]})
    
    # optimise acquisition function
    x_new, _ = slsqp(func=acq, bounds=bounds, constraints=cons, num_starts=5)

    # evaluate new point
    y_new = func(x_new)
    
    # add to data
    x_train = torch.vstack((x_train, x_new))
    y_train = torch.hstack((y_train, y_new))

    # print new best
    if y_new > torch.max(y_train[:-1]):
        print(f"New best at evaluation {len(y_train)}: \t Inputs: {x_new.numpy().reshape(dims).round(4)}, \t Outputs: {-y_new.numpy().round(4)}")

# results
best_iter = int(torch.argmax(y_train))
print(f"Evaluation: {best_iter+1} \t Solution: {float(y_train[best_iter]):.4f}")


New best at evaluation 40: 	 Inputs: [0.2426 0.2574 0.3801 0.3812 0.1807 0.6823], 	 Outputs: [-2.1706]
New best at evaluation 43: 	 Inputs: [0.2159 0.2841 0.3293 0.3916 0.2908 0.5618], 	 Outputs: [-2.4406]
New best at evaluation 46: 	 Inputs: [0.1803 0.1897 0.3561 0.3714 0.2694 0.6034], 	 Outputs: [-2.8027]
New best at evaluation 52: 	 Inputs: [0.2436 0.1538 0.4723 0.3641 0.2724 0.6077], 	 Outputs: [-2.9484]
New best at evaluation 56: 	 Inputs: [0.2045 0.1369 0.5032 0.3406 0.2733 0.6303], 	 Outputs: [-3.0979]
New best at evaluation 57: 	 Inputs: [0.208  0.1383 0.457  0.257  0.3181 0.6691], 	 Outputs: [-3.3]
New best at evaluation 65: 	 Inputs: [0.2012 0.1452 0.4682 0.2816 0.3081 0.6545], 	 Outputs: [-3.3196]
New best at evaluation 66: 	 Inputs: [0.2011 0.1455 0.4686 0.281  0.3085 0.6547], 	 Outputs: [-3.32]
New best at evaluation 67: 	 Inputs: [0.2009 0.1458 0.4684 0.2803 0.3089 0.655 ], 	 Outputs: [-3.3204]
New best at evaluation 68: 	 Inputs: [0.201  0.1471 0.4692 0.2798 0.3091 0.655